In [1]:
Pkg.update() 
Pkg.add("JuMP")
Pkg.add("GLPKMathProgInterface")

INFO: Updating METADATA...
INFO: Computing changes...
INFO: No packages to install, update or remove
INFO: Package JuMP is already installed


In [2]:
using JuMP
using MathProgBase
using GLPKMathProgInterface

INFO: Package GLPKMathProgInterface is already installed


In [3]:
include("util.jl")
include("network.jl")
include("problem.jl")

* Input and output constraints in helper file
* call function encode - aim is to transform a network, input constraints, and output constraints into a MIP
* encode(network, input_constraints, output_constraints) - return MIP

In [5]:
#=
Create example problem using small_nnet and the input/output constraitns in Reverify.ipynb
=#
small_nnet = read_nnet("small_nnet.txt")
input = Constraints()

Network(Layer[Layer([1.0; 1.0], [1.5, 1.5]), Layer([2.0 2.0; 2.0 2.0], [2.5, 2.5]), Layer([3.0 3.0], [3.5])])